# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_df = pd.read_csv('tic-tac-toe.csv')

print(data_df.head(10))

print(data_df['class'].unique())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
5  x  x  x  x  o  o  b  b  o   True
6  x  x  x  x  o  b  o  o  b   True
7  x  x  x  x  o  b  o  b  o   True
8  x  x  x  x  o  b  b  o  o   True
9  x  x  x  x  b  o  o  o  b   True
[ True False]


In [3]:
data_df.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [4]:
data_df = data_df.replace({'x' : 0, 'o' : 1, 'b' : 2})

print(data_df.head())

print(data_df.dtypes)

   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   0   0   0   0   1   1   0   1   1   True
1   0   0   0   0   1   1   1   0   1   True
2   0   0   0   0   1   1   1   1   0   True
3   0   0   0   0   1   1   1   2   2   True
4   0   0   0   0   1   1   2   1   2   True
TL       int64
TM       int64
TR       int64
ML       int64
MM       int64
MR       int64
BL       int64
BM       int64
BR       int64
class     bool
dtype: object


/var/folders/t4/gbgxnn9j3sj48qv4ydr2n_100000gn/T/ipykernel_10440/2868151260.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_df = data_df.replace({'x' : 0, 'o' : 1, 'b' : 2})


In [5]:
X = data_df.drop('class', axis = 1)
y = data_df['class']

In [6]:
X_norm = (X - X.mean() / X.std())

print(X_norm.head())

         TL        TM        TR        ML        MM        MR        BL  \
0 -1.002699 -1.084387 -1.002699 -1.084387  0.070114 -0.084387 -1.002699   
1 -1.002699 -1.084387 -1.002699 -1.084387  0.070114 -0.084387 -0.002699   
2 -1.002699 -1.084387 -1.002699 -1.084387  0.070114 -0.084387 -0.002699   
3 -1.002699 -1.084387 -1.002699 -1.084387  0.070114 -0.084387 -0.002699   
4 -1.002699 -1.084387 -1.002699 -1.084387  0.070114 -0.084387  0.997301   

         BM        BR  
0 -0.084387 -0.002699  
1 -1.084387 -0.002699  
2 -0.084387 -1.002699  
3  0.915613  0.997301  
4 -0.084387  0.997301  


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [9]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 MB 31.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 31.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 20.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 31.0 MB/s eta 0:00:00


In [7]:
# Split into training and test data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size = 0.2, random_state = 42)

In [10]:
# Create a sequential model

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

n_cols = X_norm.shape[1]

model_seq = Sequential()

# Input layer

model_seq.add(Dense(64, activation = 'relu', input_dim = (n_cols)))

# Hidden layer 1

model_seq.add(Dense(32, activation = 'relu'))

# Output layer

model_seq.add(Dense(2, activation = 'softmax'))

# Compile the model

model_seq.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Matplotlib is building the font cache; this may take a moment.
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Fit the training data

model_seq.fit(X_norm, y, epochs = 20, verbose = 2)

Epoch 1/20
30/30 - 0s - 11ms/step - accuracy: 0.6033 - loss: 0.6511
Epoch 2/20
30/30 - 0s - 1ms/step - accuracy: 0.6701 - loss: 0.5958
Epoch 3/20
30/30 - 0s - 937us/step - accuracy: 0.7056 - loss: 0.5647
Epoch 4/20
30/30 - 0s - 783us/step - accuracy: 0.7359 - loss: 0.5396
Epoch 5/20
30/30 - 0s - 807us/step - accuracy: 0.7568 - loss: 0.5153
Epoch 6/20
30/30 - 0s - 783us/step - accuracy: 0.7797 - loss: 0.4936
Epoch 7/20
30/30 - 0s - 803us/step - accuracy: 0.7965 - loss: 0.4692
Epoch 8/20
30/30 - 0s - 938us/step - accuracy: 0.8079 - loss: 0.4492
Epoch 9/20
30/30 - 0s - 832us/step - accuracy: 0.8257 - loss: 0.4278
Epoch 10/20
30/30 - 0s - 813us/step - accuracy: 0.8518 - loss: 0.4044
Epoch 11/20
30/30 - 0s - 835us/step - accuracy: 0.8466 - loss: 0.3864
Epoch 12/20
30/30 - 0s - 787us/step - accuracy: 0.8601 - loss: 0.3697
Epoch 13/20
30/30 - 0s - 772us/step - accuracy: 0.8716 - loss: 0.3520
Epoch 14/20
30/30 - 0s - 781us/step - accuracy: 0.8685 - loss: 0.3402
Epoch 15/20
30/30 - 0s - 778us/s

In [12]:
loss, accuracy = model_seq.evaluate(X_test, y_test)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8862 - loss: 0.2485 
Loss: 0.2465, Accuracy: 0.8958


In [13]:
# Saving model

model_seq.save('model_tic-tac-toe.keras')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [14]:
from tensorflow.keras.models import load_model

loaded_model = load_model('model_tic-tac-toe.keras')

In [15]:
# Select specific rows
sample_rows = X_test.iloc[[0, 5, 10]]
sample_labels = y_test.iloc[[0, 5, 10]]

# Make predictions
predictions = loaded_model.predict(sample_rows)
predicted_labels = (predictions > 0.5).astype(int)  # For binary classificatiion

# Print the results
print("Actual Labels:    ", list(sample_labels))
print("Predicted Labels: ", list(predicted_labels))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Actual Labels:     [False, True, True]
Predicted Labels:  [array([0, 1]), array([0, 1]), array([0, 1])]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [16]:
# Create a custom Adam optimizer with a specified learning rate

import tensorflow as tf

model_seq_opt = Sequential([
    Dense(64, activation='relu', input_dim=(n_cols)),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
])

learning_rate = 0.0005 
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model_seq_opt.compile(optimizer=custom_optimizer,
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the new model
model_seq_opt.fit(X_norm, y, epochs=30, verbose=2)

# Evaluate the model
loss, accuracy = model_seq_opt.evaluate(X_test, y_test)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

Epoch 1/30


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 - 0s - 9ms/step - accuracy: 0.5825 - loss: 0.6608
Epoch 2/30
30/30 - 0s - 950us/step - accuracy: 0.6597 - loss: 0.6177
Epoch 3/30
30/30 - 0s - 905us/step - accuracy: 0.6670 - loss: 0.5974
Epoch 4/30
30/30 - 0s - 811us/step - accuracy: 0.6942 - loss: 0.5796
Epoch 5/30
30/30 - 0s - 796us/step - accuracy: 0.7140 - loss: 0.5642
Epoch 6/30
30/30 - 0s - 790us/step - accuracy: 0.7443 - loss: 0.5501
Epoch 7/30
30/30 - 0s - 788us/step - accuracy: 0.7484 - loss: 0.5359
Epoch 8/30
30/30 - 0s - 776us/step - accuracy: 0.7526 - loss: 0.5220
Epoch 9/30
30/30 - 0s - 813us/step - accuracy: 0.7651 - loss: 0.5090
Epoch 10/30
30/30 - 0s - 802us/step - accuracy: 0.7808 - loss: 0.4966
Epoch 11/30
30/30 - 0s - 807us/step - accuracy: 0.7891 - loss: 0.4833
Epoch 12/30
30/30 - 0s - 1ms/step - accuracy: 0.8048 - loss: 0.4711
Epoch 13/30
30/30 - 0s - 865us/step - accuracy: 0.8038 - loss: 0.4581
Epoch 14/30
30/30 - 0s - 1ms/step - accuracy: 0.8257 - loss: 0.4465
Epoch 15/30
30/30 - 0s - 787us/step - accuracy

**Which approach(es) did you find helpful to improve your model performance?**

In [17]:
# running more epochs worked best